In [1]:
# imports
import os
from pathlib import Path
from IPython.display import Markdown, display, update_display
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# models
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [9]:
# set up environment
class CodingAssistant:
    system_prompt = "You are an assistant that analyzes or creates or help correcting \
                       python based codes. You are also expert in answering python programming \
                       related problems.\
                       Respond your anwers in markdown with code or code snippets provided within ``` ``` marks.\
                       "
    def __init__(self, model, api_key, is_openai_api=False, stream=False):
        self.model = model
        self.api_key = api_key
        self.is_openai_api = is_openai_api
        self.stream = stream

    def initialize_openai_api(self):
        load_dotenv(override=True)
        api_key = os.getenv(self.api_key)

        # Check the key
        if not api_key:
            print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
        elif not api_key.startswith("sk-proj-"):
            print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
        elif api_key.strip() != api_key:
            print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
        else:
            print("API key found and looks good so far!")

    def message_for_llm(self, question):
        return [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": question}
        ]


    def static_response(self, question):
        if self.is_openai_api:
            self.initialize_openai_api()
            openai = OpenAI()
        else:
            openai = OpenAI(base_url='http://localhost:11434/v1', api_key=self.api_key)

        response = openai.chat.completions.create(
        model = self.model,   # Example "gpt-4o-mini"
        messages = self.message_for_llm(question))
        return response.choices[0].message.content

    def stream_response(self, question):
        if self.is_openai_api:
            self.initialize_openai_api()
            openai = OpenAI()
        else:
            openai = OpenAI(base_url='http://localhost:11434/v1', api_key=self.api_key)

        stream = openai.chat.completions.create(model=self.model,
                                                messages=[{"role": "system", "content": self.system_prompt},
                                                          {"role": "user", "content": question}
                                                         ],
                                                stream=True
                                               )
        response = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            # response = response.replace("```","").replace("markdown", "")
            update_display(Markdown(response), display_id=display_handle.display_id)


    def ask(self, question):
        if self.stream:
            self.stream_response(question)

        else:
            response = self.static_response(question)
            display(Markdown(response))

In [10]:
assistant = CodingAssistant(model=MODEL_GPT, api_key="OPENAI_API_KEY",
                            is_openai_api=True, stream=True)

In [12]:
assistant.ask(question="Give important python dunder examples and methods")

API key found and looks good so far!


Dunder methods, or "double underscore" methods, are special methods in Python that allow developers to define the behavior of certain operations on objects. They are also known as "magic methods." Here are some important dunder methods along with examples to illustrate their usage:

### 1. `__init__`
The initializer method, called when an instance of a class is created.

```python
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

p = Point(3, 4)
print(p.x, p.y)  # Output: 3 4
```

### 2. `__str__`
Defines a string representation of an object when using `print()` or `str()`.

```python
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __str__(self):
        return f"Point({self.x}, {self.y})"

p = Point(3, 4)
print(p)  # Output: Point(3, 4)
```

### 3. `__repr__`
Defines an "official" string representation of an object, typically used for debugging.

```python
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f"Point({self.x}, {self.y})"

p = Point(3, 4)
print(repr(p))  # Output: Point(3, 4)
```

### 4. `__len__`
Allows an object to respond to the `len()` function.

```python
class MyList:
    def __init__(self, items):
        self.items = items

    def __len__(self):
        return len(self.items)

my_list = MyList([1, 2, 3, 4])
print(len(my_list))  # Output: 4
```

### 5. `__getitem__`
Allows indexing into an object using square brackets.

```python
class MyList:
    def __init__(self, items):
        self.items = items

    def __getitem__(self, index):
        return self.items[index]

my_list = MyList([10, 20, 30])
print(my_list[1])  # Output: 20
```

### 6. `__setitem__`
Allows assignment to an index in an object.

```python
class MyList:
    def __init__(self):
        self.items = []

    def __setitem__(self, index, value):
        self.items.insert(index, value)

my_list = MyList()
my_list[0] = 10
my_list[1] = 20
print(my_list.items)  # Output: [10, 20]
```

### 7. `__add__`
Defines behavior for the addition operator `+`.

```python
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

v1 = Vector(2, 3)
v2 = Vector(5, 7)
v3 = v1 + v2
print(v3.x, v3.y)  # Output: 7 10
```

### 8. `__eq__`
Defines behavior for equality comparisons using `==`.

```python
class Person:
    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return self.name == other.name

p1 = Person("Alice")
p2 = Person("Alice")
p3 = Person("Bob")
print(p1 == p2)  # Output: True
print(p1 == p3)  # Output: False
```

### 9. `__iter__`
Allows an object to be iterable.

```python
class MyRange:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        
    def __iter__(self):
        self.current = self.start
        return self
    
    def __next__(self):
        if self.current < self.end:
            result = self.current
            self.current += 1
            return result
        else:
            raise StopIteration

for num in MyRange(1, 5):
    print(num)  # Output: 1 2 3 4
```

### 10. `__call__`
Allows an instance of a class to be called as a function.

```python
class Multiplier:
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, x):
        return x * self.factor

double = Multiplier(2)
print(double(5))  # Output: 10
```

These examples highlight the versatility and power of dunder methods for customizing object behavior in Python!

In [15]:
ollama_assistant = CodingAssistant(model=MODEL_LLAMA, api_key="ollama",
                                   is_openai_api=False, stream=True)

In [16]:
ollama_assistant.ask(question="Give important python dunder examples and methods")

**Python Dunder Methods**
==========================

Dunder methods, also known as "magic methods," are special methods in Python classes that provide a specific functionality when called. These methods are denoted by double underscores (`__`) on either side of the method name.

### Class Example
```python
class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __add__(self, other):
        """Return a new Vector with the coordinates added"""
        return Vector(self.x + other.x, self.y + other.y)

    def __str__(self):
        """Return a string representation of the Vector"""
        return f"Vector({self.x}, {self.y})"

# Usage
v1 = Vector(2, 3)
v2 = Vector(4, 5)

print(v1 + v2)  # Output: Vector(6, 8)
```
In this example, `__add__` is a dunder method that returns a new `Vector` object with the coordinates added.

### Example Dunder Methods
-------------------------

Here are some common Python dunder methods:

#### **Initialization**

* `__init__(self, ...)`: Initializes the object's attributes.
* `__new__(cls, ...)`: Creates a new instance of the class.

#### **String Conversion**

* `__str__(self)`: Returns a string representation of the object.
* `__repr__(self)`: Returns a string representation of the object that can be used to recreate it.

```python
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __str__(self):
        return f"{self.name} is {self.age} years old."

p = Person("John", 30)
print(p)  # Output: John is 30 years old.
```

#### **Container Operations**

* `__getitem__(self, index)`: Returns the element at the specified index (e.g., array indexing).
* `__setitem__(self, index, value)`: Sets the element at the specified index to the given value.
* `__len__(self)`: Returns the length of the object.

```python
class ListLike:
    def __init__(self, elements):
        self.elements = elements

    def __getitem__(self, index):
        return self.elements[index]

# Usage
ll = ListLike([1, 2, 3])
print(ll[0])  # Output: 1
```

#### **Comparison**

* `__eq__(self, other)`: Returns `True` if the object is equal to the given object.
* `__lt__(self, other)` etc.

```python
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

p1 = Person("John", 30)
p2 = Person("John", 30)

print(p1 == p2)  # Output: True
```

#### **Container Iteration**

* `__iter__(self)` returns an iterator object.
* `__next__(self)` is the method called by the iterator to retrieve the next value.

```python
class NumberIterator:
    def __init__(self, start, end):
        self.start = start
        self.end = end

    def __iter__(self):
        return self

    def __next__(self):
        if self.start < self.end:
            result = self.start
            self.start += 1
            return result
        else:
            raise StopIteration

# Usage
num_iter = NumberIterator(0, 10)

for num in num_iter:
    print(num)  # Output: 0, 1, ..., 9
```

Note that these are just a few examples of Python dunder methods. There are many more!